Script to calculate return periods of the observed water level data. Currently uses the complete time series of each station and is not restricted to the 2003-2023 period. 

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import genextreme
import os

In [2]:
# define country and directory
country = 'mozambique'
directory = '/s3/scratch/jamie.towner/flood_aa'

In [10]:
# define paths to data
observed_data_directory = os.path.join(directory, country, "data/observations/gauging_stations/all_stations")
observed_data_file = "observations_complete_series.csv" # use observations.csv for 2003-2023 period

# load data
observed_data_path = os.path.join(observed_data_directory, observed_data_file)
observed_data = pd.read_csv(observed_data_path)

In [11]:
# convert date columns to datetime
observed_data["date"] = pd.to_datetime(observed_data["date"], format='mixed')

In [27]:
# function to calculate return periods for a given station's data
def calculate_return_periods(station_data, years=[2, 5, 10]):
    # drop NA values
    station_data = station_data.dropna().copy()

    # extract the year from the date column 
    station_data.loc[:, 'year'] = station_data['date'].dt.year
    
    # group by year and get the maximum value for each year 
    annual_max = station_data.groupby('year')[station_data.columns[1]].max()
    
    # fit the data to a GEV distribution (Generalized Extreme Value distribution)
    #params = genextreme.fit(annual_max)

    # calculate the return period for each year 
    return_periods = {}
    for return_year in years:
        # the formula for return period is: 1 / (1 - F(x))
        # F(x) is the CDF of the fitted distribution at the threshold (max value)
        threshold = np.percentile(annual_max, 100 * (1 - 1/return_year))
        #threshold = genextreme.ppf(1 - 1/return_year, *params)
        return_periods[return_year] = threshold

    return return_periods

# initialize a dictionary to store return periods for each station
return_periods_dict = {}

# iterate over each station in the observed_data 
for station in observed_data.columns:
    if station == 'date':
        continue  # Skip 'date' column

    # get the data for this station
    station_data = observed_data[['date', station]]

    # calculate return periods for the station
    return_periods = calculate_return_periods(station_data)
    
    # store the return periods in the dictionary
    return_periods_dict[station] = return_periods

# convert the dictionary to a dataframe
return_periods_df = pd.DataFrame.from_dict(return_periods_dict, orient='index')

In [28]:
return_periods_df

,2,5,10
gurue,4.066667,4.545333,4.884000
goonda,5.800000,7.196000,8.444000
messalo_em_chai,5.295000,5.700000,5.796667
revue,2.350000,2.646667,2.872000
franca,4.863333,7.112000,7.623000
mocuba,6.563333,7.626667,8.486000
nairoto,4.396667,4.990000,5.184000
massangena,4.910000,5.357333,5.873333
espungabera,3.113333,3.800000,4.210000
dombe,7.973333,9.373333,9.794000
